In [1]:
import pickle
import requests
import ipywidgets as widgets

def print_map(a,lvl=0):
    for key, val in a.items():
        indent = ''
        for i in range(lvl):
            indent = indent + '  '
        if type(val) is dict:
            print(indent,'-', key)
            print_map(val,lvl+1)
        else:
            print(indent,'-', key, ':', val)

with open('triumphs.data', 'rb') as filehandle:
    triumphs = pickle.load(filehandle)
    
with open('bounties.data', 'rb') as filehandle:
    bounties = pickle.load(filehandle)
    
with open('requirements_tasks.data', 'rb') as filehandle:
    requirements = pickle.load(filehandle)

output = widgets.Output()
display(output)

Output()

In [2]:
gamertag = 'skalligman'

#retrieve membershipId
HEADERS = {'X-API-KEY':'ae7eca0de2cf4bdea9777dc66d862f71'}
base = 'https://www.bungie.net/Platform/Destiny2'

#retrieve membershipId
URL = f'{base}/SearchDestinyPlayer/-1/{gamertag}/'
r = requests.get(url = URL, headers = HEADERS) 
data = r.json()
membershipId = data['Response'][0]['membershipId']

#retrieve characterIds
URL = f'{base}/1/Profile/{membershipId}/?components=200'
r = requests.get(url = URL, headers = HEADERS) 
data = r.json()
characterIds = list(data['Response']['characters']['data'].keys())
characters = {}
for characterId in characterIds:
    if data['Response']['characters']['data'][characterId]['classHash'] == 2271682572:
        characters[characterId] = 'Warlock' 
    if data['Response']['characters']['data'][characterId]['classHash'] == 671679327:
        characters[characterId] = 'Hunter' 
    if data['Response']['characters']['data'][characterId]['classHash'] == 3655393761:
        characters[characterId] = 'Titan' 

URL = f'{base}/1/Profile/{membershipId}/?components=900'
r = requests.get(url = URL, headers = HEADERS) 
data = r.json()
records = data['Response']['profileRecords']['data']['records']
for characterId in characterIds:
    records.update(data['Response']['characterRecords']['data'][characterId]['records'])
for record in records:
    if int(record) in triumphs:
        if 'objectives' in records[record]:
            obj = 'objectives'
        if 'intervalObjectives' in records[record]:
            obj = 'intervalObjectives'
        triumphs[int(record)]['complete'] = True
        for objective in records[record][obj]:
            objectiveHash = objective['objectiveHash']
            triumphs[int(record)]['objectives'][objectiveHash]['progress'] = objective['progress']
            triumphs[int(record)]['objectives'][objectiveHash]['complete'] = objective['complete']
            if not objective['complete']:
                triumphs[int(record)]['complete'] = False

bountyInstances = {}
nonInstancedItems = {}

URL = f'{base}/1/Profile/{membershipId}?components=201'
r = requests.get(url = URL, headers = HEADERS) 
data = r.json()
for characterId in data['Response']['characterInventories']['data']:
    nonInstancedItems[characterId] = []
    for item in data['Response']['characterInventories']['data'][characterId]['items']:
        if item['bucketHash'] == 1345459588:
            if 'itemInstanceId' in item:
                if item['itemHash'] in bounties:
                    bountyInstances[item['itemInstanceId']] = bounties[item['itemHash']]
                    bountyInstances[item['itemInstanceId']]['itemHash'] = item['itemHash']
                    if 'expirationDate' in item:
                        bountyInstances[item['itemInstanceId']]['expirationDate'] = item['expirationDate']
                    else:
                        bountyInstances[item['itemInstanceId']]['expirationDate'] = 'none'
            else:
                nonInstancedItems[characterId].append(item['itemHash'])

        
                
for bountyInstance in bountyInstances:
    URL = f'{base}/1/Profile/{membershipId}/Item/{bountyInstance}/?components=301'
    r = requests.get(url = URL, headers = HEADERS)
    bountyInstanceData = r.json()
    bountyInstances[bountyInstance]['character'] = characters[bountyInstanceData['Response']['characterId']]
    for objective in bountyInstanceData['Response']['objectives']['data']['objectives']:
        bountyInstances[bountyInstance]['objectives'][objective['objectiveHash']]['progress'] = objective['progress']
        bountyInstances[bountyInstance]['objectives'][objective['objectiveHash']]['complete'] = objective['complete']



In [5]:
all_tasks = {}

for triumph in triumphs:
    all_tasks[triumph] = {
        'path': triumphs[triumph]['path'],
        'name': triumphs[triumph]['name'],
        'description': triumphs[triumph]['description'],
        'objectives': {}}
    for objective in triumphs[triumph]['objectives']:
        all_tasks[triumph]['objectives'][objective] = {
            'description': triumphs[triumph]['objectives'][objective]['description'],
            'completionValue': triumphs[triumph]['objectives'][objective]['completionValue'],
            'progress': triumphs[triumph]['objectives'][objective]['progress']
        }
        

        
for bounty in bounties:
    vendor = bounties[bounty]['vendor']
    all_tasks[bounty] = {
        'path': f'Pursuits/Bounties/{vendor}',
        'name': bounties[bounty]['name'],
        'description': bounties[bounty]['description'],
        'objectives': {}}
    for objective in bounties[bounty]['objectives']:
        all_tasks[bounty]['objectives'][objective] = {
            'description': bounties[bounty]['objectives'][objective]['description'],
            'completionValue': bounties[bounty]['objectives'][objective]['completionValue'],
            'progress': bounties[bounty]['objectives'][objective]['progress']
        }

In [6]:

hbox_GUI = widgets.HBox()


In [7]:
options = {
    'Class': ['Any', 'Titan', 'Hunter', 'Warlock'],
    'Subclass': {
        'Titan': ['Any', 'Sentinel', 'Striker', 'Sunbreaker'],
        'Hunter': ['Any', 'Nightstalker', 'Arcstrider', 'Gunslinger'],
        'Warlock': ['Any', 'Voidwalker', 'Stormcaller', 'Dawnblade']
    },
    'Tree': {
        'Titan': {
            'Sentinel': ['Any', 'Code of the Commander', 'Code of the Protector', 'Code of the Aggressor'],
            'Striker': ['Any', 'Code of the Earthshaker', 'Code of the Missile', 'Code of the Juggernaut'],
            'Sunbreaker':['Any', 'Code of the Fire-Forged', 'Code of the Devastator', 'Code of the Siegebreaker']
        },
        'Hunter': {
            'Nightstalker': ['Any', 'Way of the Trapper', 'Way of the Wraith', 'Way of the Pathfinder'],
            'Arcstrider': ['Any', 'Way of the Warrior', 'Way of the Current', 'Way of the Wind'],
            'Gunslinger': ['Any', 'Way of the Outlaw', 'Way of a Thousand Cuts', 'Way of the Outlaw']
        },
        'Warlock': {
            'Voidwalker': ['Any', 'Attunement of Chaos', 'Attunement of Fission', 'Attunement of Hunger'],
            'Stormcaller': ['Any', 'Attunement of Conduction', 'Attunement of Control', 'Attunement of the Elements'],
            'Dawnblade': ['Any', 'Attunement of Sky', 'Attunement of Grace', 'Attunement of Flame']
        }
    },
    'Grenade': {
        'Titan': {
            'Sentinel': ['Any', 'Magnetic Grenade', 'Voidwall Grenade', 'Suppressor Grenade'],
            'Striker': ['Any', 'Flashbang Grenade', 'Pulse Grenade', 'Lightning Grenade'],
            'Sunbreaker':['Any', 'Incendiary Grenade', 'Thermite Grenade', 'Fusion Grenade']
        },
        'Hunter': {
            'Nightstalker':['Any' ,'Vortex Grenade', 'Spike Grenade', ' Voidwall Grenade'],
            'Arcstrider': ['Any', 'Skip Grenade', 'Flux Grenade', 'Arcbolt Grenade'],
            'Gunslinger': ['Any', 'Incendiary Grenade', 'Swarm Grenade', 'Tripmine Grenade']
        },
        'Warlock': {
            'Voidwalker': ['Any', 'Vortex Grenade', 'Axion Bolt', 'Scatter Grenade'],
            'Stormcaller': ['Any', 'Arcbolt Grenade', 'Pulse Grenade', 'Storm Grenade'],
            'Dawnblade': ['Any', 'Solar Grenade', 'Firebolt Grenade', 'Fusion Grenade']
        }
    },
    
    'Kinetic Weapon': ['Any', 'Hand Cannon', 'Auto Rifle', 'Scout Rifle', 'Pulse Rifle', 'Sidearm', 'Submachine Gun', 'Combt Bow', 'Sniper Rifle', 'Shotgun'],
    'Energy Weapon': ['Any', 'Hand Cannon', 'Auto Rifle', 'Scout Rifle', 'Pulse Rifle', 'Sidearm', 'Submachine Gun', 'Combt Bow', 'Sniper Rifle', 'Shotgun', 'Fusion Rifle', 'Grenade Launcher'],
    'Power Weapon': ['Any', 'Machine Gun', 'Sword', 'Grenade Launcher', 'Rocket Launcher', 'Linear Fusion Rifle'],
    'Energy Weapon Type': ['Any', 'Arc', 'Solar', 'Void'],
    'Power Weapon Type': ['Any', 'Arc', 'Solar', 'Void'],
    'Armor': ['Any'],
    
    'Activity Type': ['Any', 'Patrol', 'Gambit', 'Reckoning', 'Menagerie', 'Vex Offensive', 'Raid', 'Strike', 'Nightfall', 'Crucible', 'Forge'],
    'Activity': {
        'Patrol': ['Any', 'Earth', 'Io', 'Nessus', 'Titan','Mars', 'Dreaming City', 'Tangled Shore', 'The Moon'],
        'Strike': ['Any', 'Lake of Shadows','Savathûn\'s Song','The Arms Dealer','The Inverted Spire','The Pyramidion', 'A Garden World','Tree of Probabilities','Will of The Thousands','Strange Terrain','The Insight Termius','Warden of Nothing','Broodhold','The Hollowed Lair','The Corrupted'],
        'Nightfall': ['Any', 'Lake of Shadows','Savathûn\'s Song','The Arms Dealer','The Inverted Spire','The Pyramidion', 'A Garden World','Tree of Probabilities','Will of The Thousands','Strange Terrain','The Insight Termius','Warden of Nothing','Broodhold','The Hollowed Lair','The Corrupted'],
        'Raid': ['Any', 'Liathan', 'Prestige Leviathan', 'Eater of Worlds', 'Spire of Stars', 'Last Wish', 'Scourage of The Past', 'Crown of Sorrow', 'Garden of Salvation'],
        'Crucible': ['Any', 'Clash', 'Control', 'Showdown', 'Mayhem', 'Breakthrough', 'Lockdown', 'Rumble', 'Supremacy', 'Doubles', 'Scorched', 'Countdown', 'Survival', 'Iron Banner'],
        'Gambit': ['Any', 'Gambit', 'Gambit Prime'],
        'Reckoning': ['Any', 'Tier I', 'Tier II', 'Tier III'],
        'Menagerie': ['Any', 'Normal', 'Heroic'],
        'Forge': ['Any', 'Volundr Forge', 'Gofannon Forge', 'Izanami Forge', 'Bergusia Forge']
    }
}


#build Options Pane

box_spacer = widgets.Box(layout=widgets.Layout(height='20px'))

#---------------------Filter---------------------
layout_box = widgets.Layout(
    border='1px solid white',
    width='15%',
    padding='10px'
)

layout_general = widgets.Layout(
    width='98%',
    height = '100%'
)

layout_dropdown_inactive = widgets.Layout(
    width='98%',
    height = '100%',
    border = ''
)

layout_dropdown_active = widgets.Layout(
    width='98%',
    border = '2px solid #00bcd4',
    height = '100%'
)


#---------------------Class---------------------
label_class_options = widgets.HTML(value = "<b>Class Options</b>")

label_class = widgets.Label(value='Class:',layout = layout_general)
dropdown_class = widgets.Dropdown(options = options['Class'],layout = layout_dropdown_inactive)

label_subclass = widgets.Label(value='Subclass:',layout = layout_general)
dropdown_subclass = widgets.Dropdown(options = ['Any'],disabled = True,layout = layout_dropdown_inactive)

label_tree = widgets.Label(value='Tree:',layout = layout_general)
dropdown_tree = widgets.Dropdown(options = ['Any'],disabled = True,layout = layout_dropdown_inactive)

label_grenade = widgets.Label(value='Grenade:',layout = layout_general)
dropdown_grenade = widgets.Dropdown(options = ['Any'],disabled = True,layout = layout_dropdown_inactive)

vbox_class_options = widgets.VBox(
    children = [
        label_class_options,
        label_class,
        dropdown_class,
        label_subclass,
        dropdown_subclass,
        label_tree,
        dropdown_tree,
        label_grenade,
        dropdown_grenade])


#---------------------Gear---------------------
label_gear_options = widgets.HTML(value = "<b>Gear Options</b>")

label_kinetic = widgets.Label(value='Kinetic Weapon:',layout = layout_general)
dropdown_kinetic = widgets.Dropdown(options = options['Kinetic Weapon'],layout = layout_dropdown_inactive)

label_energy = widgets.Label(value='Energy Weapon:',layout = layout_general)
dropdown_energy = widgets.Dropdown(options = options['Energy Weapon'],layout = layout_general)
dropdown_energy_type = widgets.Dropdown(options = options['Energy Weapon Type'],layout = layout_dropdown_inactive)

label_power = widgets.Label(value='Power Weapon:',layout = layout_general)
dropdown_power = widgets.Dropdown(options = options['Power Weapon'])
dropdown_power_type = widgets.Dropdown(options = options['Power Weapon Type'],layout = layout_dropdown_inactive)

label_armor = widgets.Label(value='Armor Set:',layout = layout_general)
dropdown_armor = widgets.Dropdown(options = options['Armor'],layout = layout_dropdown_inactive)

vbox_gear = widgets.VBox(
    children = [
        label_gear_options,
        label_kinetic,
        dropdown_kinetic,
        label_energy,
        dropdown_energy,
        dropdown_energy_type,
        label_power,
        dropdown_power,
        dropdown_power_type,
        label_armor,
        dropdown_armor])

#---------------------Activities---------------------
label_activity_options = widgets.HTML(
    value = "<b>Activity Options</b>",layout = layout_general)

dropdown_activity_type = widgets.Dropdown(options = options['Activity Type'],layout = layout_dropdown_inactive)

dropdown_activity = widgets.Dropdown(options = ['Any'],disabled = True,layout = layout_dropdown_inactive)

vbox_activity = widgets.VBox(
    children=[
        label_activity_options,
        dropdown_activity_type,
        dropdown_activity])

#----------Options-------------

vbox_options = widgets.VBox(
    children=[
        vbox_activity,
        box_spacer,
        vbox_class_options,
        box_spacer,
        vbox_gear],
    layout = layout_box)

#-----------------------Interact Functions --------------------------

def change_class(change):
    if change['new'] == 'Any':
        dropdown_subclass.options = ['Any']
        dropdown_subclass.disabled = True
    else:
        dropdown_subclass.options = options['Subclass'][change['new']]
        dropdown_subclass.disabled = False
        
def change_subclass(change):
    if change['new'] == 'Any':
        dropdown_tree.options = ['Any']
        dropdown_tree.disabled = True
        dropdown_grenade.options = ['Any']
        dropdown_grenade.disabled = True
    else:
        dropdown_tree.options = options['Tree'][dropdown_class.value][change['new']]
        dropdown_tree.disabled = False
        dropdown_grenade.options = options['Grenade'][dropdown_class.value][change['new']]
        dropdown_grenade.disabled = False
        
def change_activity_type(change):
    if change['new'] == 'Any':
        dropdown_activity.options = ['Any']
        dropdown_activity.disabled = True
    else:
        dropdown_activity.options = options['Activity'][change['new']]
        dropdown_activity.disabled = False
        
        
dropdown_class.observe(change_class, names = 'value')
dropdown_subclass.observe(change_subclass, names = 'value')
dropdown_activity_type.observe(change_activity_type, names = 'value')


In [8]:
n_per_page = 15

def toggle_objective(b):
    if b.objective_hash in active_objectives:
        active_objectives.remove(b.objective_hash)
    else:
        active_objectives.append(b.objective_hash)
    refresh_loadout()
    refresh_task_view()
    
    
    
def edit_objective(b):
    dropdown_triumph_req.value = b.task_hash
    dropdown_objective_req.value = b.objective_hash

def build_task_widget():

    layout_vbox_triumph = widgets.Layout(
        margin = '10px',
        padding='10px',
        border='3px solid white',
        width = '400px',
        overflow = 'visible')

    layout_label_name = widgets.Layout(
        width = '100%',
        overflow = 'visible')

    layout_label_word = widgets.Layout(
        )

    layout_hbox_description = widgets.Layout(
        display = 'flex',
        flex_flow = 'row wrap')

    layout_vbox_objectives = widgets.Layout(
        width = '100%',
        overflow = 'visible')


    layout_label_objective = widgets.Layout(
        padding='1px',
        width = '0px',
        overflow = 'visible',
        order='-1')

    layout_intProgress_objective = widgets.Layout(
        padding='0px',
        #border='1px solid black',
        width = '100%',
        height = '32px',
        overflow = 'visible')

    layout_button_objective = widgets.Layout(
        width = '32px',
        height = '32px',
        order = '-2')

    vbox_triumph = widgets.VBox(layout=layout_vbox_triumph)
    
    label_path = widgets.HTML(value = '<font size="1">Path/Path/Path/Path/</font size>', layout = layout_label_name)

    label_name = widgets.HTML(value = '<b>Task Name</b>', layout = layout_label_name)

    label_description = widgets.HTML(value = '<i>test test test test test test test test test test test test test test test test test test test test test test test test </i>')

    vbox_objectives = widgets.VBox(layout=layout_vbox_objectives)
    objectives = []
    for i in range(6):
        
        button_objective = widgets.Button(layout = layout_button_objective)
        button_objective.objective_hash = 0
        button_objective.on_click(toggle_objective)
        button_edit = widgets.Button(description='EDIT', layout=widgets.Layout(width='60px'))
        button_edit.on_click(edit_objective)
        
        objective_description = f'objective {i}'
        label_objective = widgets.Label(value = f'\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0{objective_description}', layout = layout_label_objective)
        
        intProgress_objective = widgets.IntProgress(
            value=30,
            min=0,
            max=100,
            step=1,
            layout = layout_intProgress_objective)
        
        hbox_objective = widgets.HBox()
        hbox_objective.children = [button_objective,intProgress_objective, label_objective, button_edit]
        button_objective.parent = hbox_objective
        hbox_objective.parent = vbox_objectives
        
        
        objectives.append(hbox_objective)

    vbox_objectives.children = objectives
    vbox_triumph.children = [label_path,label_name,label_description, vbox_objectives]
    vbox_objectives.parent = vbox_triumph

    return (vbox_triumph)
    
vbox_tasks = widgets.VBox(layout=widgets.Layout(width='85%'))

hbox_tasks_header = widgets.HBox(layout=widgets.Layout(justify_content = 'space-between'))
button_refresh = widgets.Button(description='Refresh')
button_previous_page = widgets.Button(description = 'Previous Page')
button_next_page = widgets.Button(description = 'Next Page')
label_current_page = widgets.HTML('Current Page')
hbox_task_buttons = widgets.HBox(children=[button_previous_page,button_next_page])
vbox_task_info = widgets.VBox(children=[label_current_page,hbox_task_buttons])
checkbox_complete = widgets.Checkbox(description='Hide Complete')
checkbox_triumphs = widgets.Checkbox(description='Hide Triumphs')
checkbox_pursuits = widgets.Checkbox(description='Hide Pursuits')
checkbox_collections = widgets.Checkbox(description='Hide Collections')
checkbox_milestones = widgets.Checkbox(description='Hide Milestones')
checkbox_conflicting = widgets.Checkbox(description='Hide Conflicting Tasks')
layout_checkboxes = widgets.Layout(display = 'flex', flex_flow = 'column wrap')
box_checkboxes = widgets.Box(children=[checkbox_complete,checkbox_triumphs,checkbox_pursuits,checkbox_collections,checkbox_milestones,checkbox_conflicting],layout=layout_checkboxes)
hbox_tasks_header.children = [button_refresh,box_checkboxes,vbox_task_info]

layout_tasks = widgets.Layout(
    display = 'flex',
    flex_flow = 'row wrap',
    justify_content = 'space-between'
)
task_widgets = []
for i in range(n_per_page):
    task_widgets.append(build_task_widget())
box_tasks = widgets.Box(children=task_widgets, layout=layout_tasks)


vbox_tasks.children = [hbox_tasks_header,box_tasks]

hbox_GUI.children=[vbox_options,vbox_tasks]

In [9]:

active_objectives = []
total_potential_tasks = 0
tasks_per_page = 15
total_task_pages = 0
current_page = 0


def filter_tasks():
    
    global potential_tasks
    global total_potential_tasks
    global total_task_pages
    global current_page
    
    current_page = 0
    
    potential_tasks = []

    for task in all_tasks:
        add_task = True
        
        #filter compllete
        if checkbox_complete.value:
            complete = True
            for objective in all_tasks[task]['objectives']:
                if all_tasks[task]['objectives'][objective]['progress'] < all_tasks[task]['objectives'][objective]['completionValue']:
                    complete = False
            if complete:
                add_task = False
                
        #filter triumphs
        if checkbox_triumphs.value and ('Triumphs' in all_tasks[task]['path'] or 'Seals' in all_tasks[task]['path']):
            add_task = False
            
        #filter pursuits
        if checkbox_pursuits.value and 'Pursuits' in all_tasks[task]['path']:
            add_task = False
        
        
        if checkbox_conflicting.value:
            conflicting = True
            for objective in all_tasks[task]['objectives']:
                if not has_conflict(objective):
                    conflicting = False
            if conflicting:
                add_task = False
        if add_task:
            potential_tasks.append(task) 
 
    total_potential_tasks = len(potential_tasks)      
    total_task_pages = int(total_potential_tasks/tasks_per_page)
    if total_potential_tasks%tasks_per_page > 0:
        total_task_pages += 1



filter_tasks()



def has_conflict(objective):
    conflict = False
    if objective in requirements:
        for requirement in requirements[objective]:
            if requirements[objective][requirement] != 'Any':
                if requirement == 'Activity Type' and dropdown_activity_type.value != requirements[objective][requirement]:
                    if dropdown_activity_type.value != 'Any':
                        conflict = True
                if requirement == 'Activity' and dropdown_activity.value != requirements[objective][requirement]:
                    if dropdown_activity.value != 'Any':
                        conflict = True
                if requirement == 'Class' and dropdown_class.value != requirements[objective][requirement]:
                    if dropdown_class.value != 'Any':
                        conflict = True
                if requirement == 'Subclass' and dropdown_subclass.value != requirements[objective][requirement]:
                    if dropdown_subclass.value != 'Any':
                        conflict = True
                if requirement == 'Tree' and dropdown_tree.value != requirements[objective][requirement]:
                    if dropdown_tree.value != 'Any':
                        conflict = True
                if requirement == 'Grenade' and dropdown_grenade.value != requirements[objective][requirement]:
                    if dropdown_grenade.value != 'Any':
                        conflict = True
 
    return conflict



def refresh_task_view():
    
    global current_page
    
    if current_page == 0:
        button_previous_page.disabled = True
    else:
        button_previous_page.disabled = False
    if current_page == total_task_pages - 1:
        button_next_page.disabled = True
    else:
        button_next_page.disabled = False
    
    starting_task = current_page*tasks_per_page
    
    if total_potential_tasks == 0:
        hbox_GUI.children[1].children[0].children[2].children[0].value = 'No tasks to show!!'
    elif starting_task+tasks_per_page > total_potential_tasks:
        hbox_GUI.children[1].children[0].children[2].children[0].value = f'Showing Tasks {starting_task+1}-{total_potential_tasks} of {total_potential_tasks}'
    else:
        hbox_GUI.children[1].children[0].children[2].children[0].value = f'Showing Tasks {starting_task+1}-{starting_task+tasks_per_page} of {total_potential_tasks}'

    for i in range(tasks_per_page):
        
        #hide extra task boxes
        if starting_task + i >= total_potential_tasks:
            hbox_GUI.children[1].children[1].children[i].layout.display = 'none'
        else:
            hbox_GUI.children[1].children[1].children[i].layout.display = 'flex'
            task = potential_tasks[starting_task+i]
            
        
            #update values in task box
            task_path = all_tasks[task]['path']
            hbox_GUI.children[1].children[1].children[i].children[0].value = f'<font size="1">{task_path}/</font size=>'

            task_name = all_tasks[task]['name']
            hbox_GUI.children[1].children[1].children[i].children[1].value = f'<font size="5"><b>{task_name}</b></font size=>'

            task_description = all_tasks[task]['description']
            hbox_GUI.children[1].children[1].children[i].children[2].value = f'<i>{task_description}</i>'

            for j in range(6):
                if j >= len(all_tasks[task]['objectives']):
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].layout.display = 'none'
                else:
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].layout.display = 'flex'

                    objective_hash = list(all_tasks[task]['objectives'].keys())[j]

                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].objective_hash = objective_hash
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[3].objective_hash = objective_hash
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[3].task_hash = task

                    objective_completion_value = all_tasks[task]['objectives'][objective_hash]['completionValue']
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].max = objective_completion_value

                    objective_progress = all_tasks[task]['objectives'][objective_hash]['progress']
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].value = objective_progress

                    objective_description = all_tasks[task]['objectives'][objective_hash]['description']
                    indent = '\xa0\xa0\xa0\xa0'
                    hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[2].value = f'{indent}{objective_description} ({objective_progress}/{objective_completion_value})'


                    #set objective color
                    if objective_progress >= objective_completion_value:
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].button_style = 'success'
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].disabled = True
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].bar_style = 'success'
                    elif objective_hash in active_objectives:
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].button_style = 'info'
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].disabled = False
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].bar_style = 'info'
                    elif has_conflict(objective_hash):
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].button_style = 'danger'
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].disabled = True
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].bar_style = 'danger'
                    else:
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].button_style = ''
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[0].disabled = False
                        hbox_GUI.children[1].children[1].children[i].children[3].children[j].children[1].bar_style = ''



            
refresh_task_view()

def previous_page(b):
    global current_page    
    current_page -= 1
    refresh_task_view()


def next_page(b):
    global current_page
    current_page += 1
    refresh_task_view()

button_previous_page.on_click(previous_page)
button_next_page.on_click(next_page)


def refresh_button(b):
    filter_tasks()
    refresh_task_view()
    
button_refresh.on_click(refresh_button)


In [10]:
possible_requirements = {
    'Class': ['Any', 'Titan', 'Hunter', 'Warlock'],
    'Subclass': {
        'Titan': ['Any', 'Sentinel', 'Striker', 'Sunbreaker'],
        'Hunter': ['Any', 'Nightstalker', 'Arcstrider', 'Gunslinger'],
        'Warlock': ['Any', 'Voidwalker', 'Stormcaller', 'Dawnblade']
    },
    'Tree': {
        'Titan': {
            'Sentinel': ['Any', 'Code of the Commander', 'Code of the Protector', 'Code of the Aggressor'],
            'Striker': ['Any', 'Code of the Earthshaker', 'Code of the Missile', 'Code of the Juggernaut'],
            'Sunbreaker':['Any', 'Code of the Fire-Forged', 'Code of the Devastator', 'Code of the Siegebreaker']
        },
        'Hunter': {
            'Nightstalker': ['Any', 'Way of the Trapper', 'Way of the Wraith', 'Way of the Pathfinder'],
            'Arcstrider': ['Any', 'Way of the Warrior', 'Way of the Current', 'Way of the Wind'],
            'Gunslinger': ['Any', 'Way of the Outlaw', 'Way of a Thousand Cuts', 'Way of the Outlaw']
        },
        'Warlock': {
            'Voidwalker': ['Any', 'Attunement of Chaos', 'Attunement of Fission', 'Attunement of Hunger'],
            'Stormcaller': ['Any', 'Attunement of Conduction', 'Attunement of Control', 'Attunement of the Elements'],
            'Dawnblade': ['Any', 'Attunement of Sky', 'Attunement of Grace', 'Attunement of Flame']
        }
    },
    'Grenade': {
        'Titan': {
            'Sentinel': ['Any', 'Magnetic Grenade', 'Voidwall Grenade', 'Suppressor Grenade'],
            'Striker': ['Any', 'Flashbang Grenade', 'Pulse Grenade', 'Lightning Grenade'],
            'Sunbreaker':['Any', 'Incendiary Grenade', 'Thermite Grenade', 'Fusion Grenade']
        },
        'Hunter': {
            'Nightstalker':['Any' ,'Vortex Grenade', 'Spike Grenade', ' Voidwall Grenade'],
            'Arcstrider': ['Any', 'Skip Grenade', 'Flux Grenade', 'Arcbolt Grenade'],
            'Gunslinger': ['Any', 'Incendiary Grenade', 'Swarm Grenade', 'Tripmine Grenade']
        },
        'Warlock': {
            'Voidwalker': ['Any', 'Vortex Grenade', 'Axion Bolt', 'Scatter Grenade'],
            'Stormcaller': ['Any', 'Arcbolt Grenade', 'Pulse Grenade', 'Storm Grenade'],
            'Dawnblade': ['Any', 'Solar Grenade', 'Firebolt Grenade', 'Fusion Grenade']
        }
    },
    'Weapon Slot': ['Any', 'Kinetic', 'Energy', 'Power'],
    'Weapon Type': ['Any', 'Hand Cannon', 'Auto Rifle', 'Scout Rifle', 'Pulse Rifle', 'Sidearm', 'Submachine Gun', 'Combt Bow','Sniper Rifle', 'Shotgun', 'Fusion Rifle', 'Grenade Launcher', 'Machine Gun', 'Sword', 'Grenade Launcher', 'Rocket Launcher', 'Linear Fusion Rifle'],
    'Weapon Ammo': ['Any', 'Primary', 'Secondary', 'Heavy'],
    'Weapon Ammo Type': {
        'Primary': ['Any', 'Hand Cannon', 'Auto Rifle', 'Scout Rifle', 'Pulse Rifle', 'Sidearm', 'Submachine Gun', 'Combt Bow',],
        'Secondary': ['Any', 'Sniper Rifle', 'Shotgun', 'Fusion Rifle', 'Grenade Launcher'],
        'Heavy': ['Any', 'Machine Gun', 'Sword', 'Grenade Launcher', 'Rocket Launcher', 'Linear Fusion Rifle']
    },
    'Weapon Energy': ['Any', 'Kinetic', 'Solar', 'Void', 'Arc'],
    'Activity Type': ['Any', 'Patrol', 'Gambit', 'Reckoning', 'Menagerie', 'Vex Offensive', 'Raid', 'Strike', 'Nightfall', 'Crucible', 'Forge'],
    'Activity': {
        'Patrol': ['Any', 'Earth', 'Io', 'Nessus', 'Titan','Mars', 'Dreaming City', 'Tangled Shore', 'The Moon'],
        'Strike': ['Any', 'Lake of Shadows','Savathûn\'s Song','The Arms Dealer','The Inverted Spire','The Pyramidion', 'A Garden World','Tree of Probabilities','Will of The Thousands','Strange Terrain','The Insight Termius','Warden of Nothing','Broodhold','The Hollowed Lair','The Corrupted'],
        'Nightfall': ['Any', 'Lake of Shadows','Savathûn\'s Song','The Arms Dealer','The Inverted Spire','The Pyramidion', 'A Garden World','Tree of Probabilities','Will of The Thousands','Strange Terrain','The Insight Termius','Warden of Nothing','Broodhold','The Hollowed Lair','The Corrupted'],
        'Raid': ['Any', 'Liathan', 'Prestige Leviathan', 'Eater of Worlds', 'Spire of Stars', 'Last Wish', 'Scourage of The Past', 'Crown of Sorrow', 'Garden of Salvation'],
        'Crucible': ['Any', 'Clash', 'Control', 'Showdown', 'Mayhem', 'Breakthrough', 'Lockdown', 'Rumble', 'Supremacy', 'Doubles', 'Scorched', 'Countdown', 'Survival', 'Iron Banner'],
        'Gambit': ['Any', 'Gambit', 'Gambit Prime'],
        'Reckoning': ['Any', 'Tier I', 'Tier II', 'Tier III'],
        'Menagerie': ['Any', 'Normal', 'Heroic'],
        'Vex Offensive': ['Any'],
        'Forge': ['Any', 'Volundr Forge', 'Gofannon Forge', 'Izanami Forge', 'Bergusia Forge']
    }
}
layout_req = widgets.Layout(width='20%')
layout_label_req = widgets.Layout(width='10%')
output_req = widgets.Output()

hbox_triumph_req = widgets.HBox()
label_triumph_req = widgets.Label("Triumph:", layout=layout_label_req)
dropdown_triumph_req = widgets.Dropdown(options=list(triumphs.keys()), layout=layout_req)
dropdown_objective_req = widgets.Dropdown(layout=layout_req)
box_spacer_req = widgets.Box(layout=layout_req)
button_save_req = widgets.Button(description='Save', layout=layout_req)
hbox_triumph_req.children = [label_triumph_req,dropdown_triumph_req,dropdown_objective_req, box_spacer_req, button_save_req]

def change_triumph(change):
    output_req.clear_output()
    with output_req: print_map(triumphs[change['new']])
    dropdown_objective_req.options = list(triumphs[change['new']]['objectives'].keys())
    

    
dropdown_triumph_req.observe(change_triumph, names='value')



#----------------------Class----------------------------


hbox_class_req = widgets.HBox()
label_class_req = widgets.Label(value='Class:',layout=layout_label_req)
dropdown_class_req = widgets.Dropdown(options=possible_requirements['Class'],layout=layout_req)
dropdown_subclass_req = widgets.Dropdown(disabled = True, options = ['Any'],layout=layout_req)
dropdown_tree_req = widgets.Dropdown(disabled = True, options = ['Any'],layout=layout_req)
dropdown_grenade_req = widgets.Dropdown(disabled = True, options = ['Any'],layout=layout_req)
hbox_class_req.children = [
    label_class_req,
    dropdown_class_req,
    dropdown_subclass_req,
    dropdown_tree_req,
    dropdown_grenade_req
]

def change_class(change):
    if change['new'] == 'Any':
        dropdown_subclass_req.options = ['Any']
        dropdown_subclass_req.disabled = True
    else:
        dropdown_subclass_req.options = possible_requirements['Subclass'][change['new']]
        dropdown_subclass_req.disabled = False
        
def change_subclass(change):
    if change['new'] == 'Any':
        dropdown_tree_req.options = ['Any']
        dropdown_tree_req.disabled = True
        dropdown_grenade_req.options = ['Any']
        dropdown_grenade_req.disabled = True
    else:
        dropdown_tree_req.options = possible_requirements['Tree'][dropdown_class_req.value][change['new']]
        dropdown_tree_req.disabled = False
        dropdown_grenade_req.options = possible_requirements['Grenade'][dropdown_class_req.value][change['new']]
        dropdown_grenade_req.disabled = False

dropdown_class_req.observe(change_class, names = 'value')
dropdown_subclass_req.observe(change_subclass, names = 'value')


#---------------Weapon-----------------------


hbox_weapon_req = widgets.HBox()
label_weapon_req = widgets.Label(value='Weapon:',layout=layout_label_req)
dropdown_weapon_slot_req = widgets.Dropdown(options = possible_requirements['Weapon Slot'],layout=layout_req)
dropdown_weapon_ammo_req = widgets.Dropdown(options = possible_requirements['Weapon Ammo'],layout=layout_req)
dropdown_weapon_type_req = widgets.Dropdown(options = possible_requirements['Weapon Type'],layout=layout_req)
dropdown_weapon_energy_req = widgets.Dropdown(options = possible_requirements['Weapon Energy'],layout=layout_req)


hbox_weapon_req.children = [
    label_weapon_req,
    dropdown_weapon_slot_req,
    dropdown_weapon_energy_req,
    dropdown_weapon_type_req,
    dropdown_weapon_ammo_req
]

stop_refresh_weapon = False

#TODO: need to add functionality to set weapon slot type

def refresh_weapon(dropdown, old, new):
    global stop_refresh_weapon
    if not stop_refresh_weapon:
        stop_refresh_weapon = True
        if dropdown == 'type':
            if new != 'Any':
                for ammo in possible_requirements['Weapon Ammo Type']:
                    if new in possible_requirements['Weapon Ammo Type'][ammo]:
                        dropdown_weapon_ammo_req.value = ammo
                        dropdown_weapon_type_req.options = possible_requirements['Weapon Ammo Type'][ammo]
                        dropdown_weapon_type_req.value = new
        elif dropdown == 'ammo':
            dropdown_weapon_type_req.value = 'Any'
            if new == 'Any':
                dropdown_weapon_type_req.options = possible_requirements['Weapon Type']
            else:
                 dropdown_weapon_type_req.options = possible_requirements['Weapon Ammo Type'][new]
        stop_refresh_weapon = False 
            

def change_weapon_type(change):
    refresh_weapon('type',change['old'],change['new'])

def change_weapon_ammo(change):
    refresh_weapon('ammo',change['old'],change['new'])

    
dropdown_weapon_type_req.observe(change_weapon_type, names = 'value')
dropdown_weapon_ammo_req.observe(change_weapon_ammo, names = 'value')


#-----------------------------Activity------------------------------


hbox_activity_req = widgets.HBox()
label_activity_req = widgets.Label(value='Activity:',layout=layout_label_req)
dropdown_activity_type_req = widgets.Dropdown(options = possible_requirements['Activity Type'],layout=layout_req)
dropdown_activity_req = widgets.Dropdown(disabled = True, options = ['Any'],layout=layout_req)

hbox_activity_req.children = [
    label_activity_req,
    dropdown_activity_type_req,
    dropdown_activity_req
]

def change_activity_type(change):
    if change['new'] == 'Any':
        dropdown_activity_req.options = ['Any']
        dropdown_activity_req.disabled = True
    else:
        dropdown_activity_req.options = possible_requirements['Activity'][change['new']]
        dropdown_activity_req.disabled = False

dropdown_activity_type_req.observe(change_activity_type, names = 'value')
        
vbox_req = widgets.VBox(children=[hbox_triumph_req,hbox_class_req,hbox_weapon_req,hbox_activity_req])



def load_objective(objective):
    if objective in requirements:
        dropdown_class_req.value = requirements[objective]['Class']
        dropdown_subclass_req.value = requirements[objective]['Subclass']
        dropdown_tree_req.value = requirements[objective]['Tree']
        dropdown_grenade_req.value = requirements[objective]['Grenade']
        dropdown_weapon_slot_req.value = requirements[objective]['Weapon Slot']
        dropdown_weapon_ammo_req.value = requirements[objective]['Weapon Ammo']
        dropdown_weapon_type_req.value = requirements[objective]['Weapon Type']
        dropdown_weapon_energy_req.value = requirements[objective]['Weapon Energy']
        dropdown_activity_type_req.value = requirements[objective]['Activity Type']
        dropdown_activity_req.value = requirements[objective]['Activity']
    else:
        dropdown_class_req.value = 'Any'
        dropdown_subclass_req.value = 'Any'
        dropdown_tree_req.value = 'Any'
        dropdown_grenade_req.value = 'Any'
        dropdown_weapon_slot_req.value = 'Any'
        dropdown_weapon_ammo_req.value = 'Any'
        dropdown_weapon_type_req.value = 'Any'
        dropdown_weapon_energy_req.value = 'Any'
        dropdown_activity_type_req.value = 'Any'
        dropdown_activity_req.value = 'Any'
    
def save_objective(b):
    objective = dropdown_objective_req.value
    if objective not in requirements:
        requirements[objective] = {}
    requirements[objective]['Class'] = dropdown_class_req.value
    requirements[objective]['Subclass'] = dropdown_subclass_req.value
    requirements[objective]['Tree'] = dropdown_tree_req.value
    requirements[objective]['Grenade'] = dropdown_grenade_req.value
    requirements[objective]['Weapon Slot'] = dropdown_weapon_slot_req.value
    requirements[objective]['Weapon Ammo'] = dropdown_weapon_ammo_req.value
    requirements[objective]['Weapon Type'] = dropdown_weapon_type_req.value
    requirements[objective]['Weapon Energy'] = dropdown_weapon_energy_req.value
    requirements[objective]['Activity Type'] = dropdown_activity_type_req.value
    requirements[objective]['Activity'] = dropdown_activity_req.value
    with open('requirements.data', 'wb') as filehandle:
        pickle.dump(requirements, filehandle) 
    

def change_objective(change):
    load_objective(change['new'])
    
dropdown_objective_req.observe(change_objective, names='value')
button_save_req.on_click(save_objective)

dropdown_triumph_req.value = list(triumphs.keys())[-1]
dropdown_triumph_req.value = list(triumphs.keys())[0]


In [11]:
display(vbox_req,hbox_GUI)

In [12]:



def set_loadout(dropdown, value):
    dropdown.value = value
    if value == 'Any':
        dropdown.disabled = False
        dropdown.layout = layout_dropdown_inactive
    else:
        dropdown.disabled = True
        dropdown.layout = layout_dropdown_active
    

loadout_list = [
    dropdown_activity_type, 
    dropdown_activity, 
    dropdown_class,
    dropdown_subclass,
    dropdown_grenade,
    dropdown_kinetic,
    dropdown_energy,
    dropdown_energy_type,
    dropdown_power,
    dropdown_power_type]


def refresh_loadout():
    
    for dropdown in loadout_list:
        set_loadout(dropdown, 'Any')
    
    for objective in active_objectives:
        for requirement in requirements[objective]:
            if requirements[objective][requirement] != 'Any':
                if requirement == 'Activity Type':
                    set_loadout(dropdown_activity_type, requirements[objective][requirement])
                if requirement == 'Activity':
                    set_loadout(dropdown_activity, requirements[objective][requirement])
                if requirement == 'Class':
                    set_loadout(dropdown_class, requirements[objective][requirement])
                if requirement == 'Subclass':
                    set_loadout(dropdown_subclass, requirements[objective][requirement])
                if requirement == 'Tree':
                    set_loadout(dropdown_tree, requirements[objective][requirement])
                if requirement == 'Grenade':
                    set_loadout(dropdown_grenade, requirements[objective][requirement])
                if requirement == 'Weapon Slot':
                    if requirements[objective][requirement] == 'Kinetic':
                        set_loadout(dropdown_kinetic, requirements[objective]['Weapon Type'])
                    if requirements[objective][requirement] == 'Energy':
                        set_loadout(dropdown_energy, requirements[objective]['Weapon Type'])
                        set_loadout(dropdown_energy_type, requirements[objective]['Weapon Energy'])
                    if requirements[objective][requirement] == 'Power':
                        set_loadout(dropdown_power, requirements[objective]['Weapon Type'])
                        set_loadout(dropdown_power_type, requirements[objective]['Weapon Energy'])

                
        
